# Workflow 3 - cluster the trajectory

**Input:** EnGen object featurized with reduced dimensionality (generated by Workflow2)


**Output:** Representative trajectory ensemble  
<hr>
Steps:

1. Import the featurized trajectory from Workflow2
2. Choose clustering technique: KMeans, Gaussian Mixture Models
3. Choose appropriate parameters for clustering (number of clusters)
4. Additionally filter trajectories
5. Extract the ensemble

In [ ]:
#required imports

from engens.core.EnGens import EnGen
from engens.core.ClustEn import *
import pickle as pk

### Step 0 - Load data from Workflow2

In [ ]:
engen = None
with open("wf2_resulting_EnGen.pickle", "rb") as file:
    engen = pk.load(file)

In [ ]:
traj = engen.traj
ref = engen.ref
print("Using the trajectory {} and reference pdb file {}".format(traj, ref))

topology = engen.mdtrajref
print("The topology is:")
print(topology)

feat_dims = engen.dimred_data.shape[0]
print("The dimensionality of your featurization is {}".format(feat_dims))

feat = engen.featurizers[engen.chosen_feat_index]
print("You chose to featurize with")
print(feat.describe())

dimred_data = engen.dimred_data
print("After dimensionality reduction the dimension of your features is {}".format(dimred_data.shape[1]))


### Step 1 - choose the clustering method

In [ ]:
#------------two clustering algorithms------------------#
#----------------choose and uncomment one----------------#
# Option 1 - choose Kmeans

clustering = "KM"
cluster_method = clusterings[clustering](engen, n_rep=2)


# Option 2 - choose GMMs
"""
clustering = "GMM" 
cluster_method = clusterings[clustering](engen, n_rep=2)
"""

### Step 2 - run the clustering with different parameters

In [ ]:
# Create K clusters
if clustering =="KM":
    params = [{"n_clusters":i} for i in range(2, 10)]
    cluster_method.cluster_multiple_params(params)
else:
    params = [{"n_components":i} for i in range(2, 10)]
    cluster_method.cluster_multiple_params(params)

# analyze these parameters with the elbow method
cluster_method.analyze_elbow_method()

In [ ]:
# analyze these parameters with the silhouette method
cluster_method.analyze_silhouette()

In [ ]:
# pick the number of clusters
n = 2
cluster_method.choose_n(n)

### Step 3 - optionally pick a subset of clusters with the heighest weight

In [ ]:
cluster_method.plot_cluster_weight()

In [ ]:
cluster_method.choose_clusters(thr=0.07)

### Step 4 - extract conformations for the ensemble

In [ ]:
ensemble_location = "./res_ensemble"
cluster_method.choose_conformations()
cluster_method.extract_conformations(ensemble_location)


### Step 5 - save results for analysis

In [ ]:
with open("wf3_resulting_EnGen.pickle", "wb") as file:
    pk.dump(engen, file, -1)
    
with open("wf3_resulting_Clust.pickle", "wb") as file:
    pk.dump(cluster_method, file, -1)